In [1]:
import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 90% !important; }</style>'))

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# CAD

In [3]:
cad = pd.read_csv("CHEERS/CHEERS_out/CAD_disease_enrichment_pValues.txt", sep='\t', header=None)
cad = cad.set_index(0)
cad.index.name = None
cad.columns = ['cad']
cad.head()

,cad
B-cell,0.092117
CD14-positive-monocyte,0.217341
CD4-positive_alpha-beta-T-cell,0.948140
astrocyte,1.000000
cardiac-muscle-cell,1.000000


In [4]:
cad = cad[~cad.index.isin(['huvec'])]
cad['CAD'] = 1-cad['cad']
cad.loc[cad['CAD'] <= 0.001, 'CAD'] = 0.001

In [5]:
cad

,cad,CAD
B-cell,0.092117,0.907883
CD14-positive-monocyte,0.217341,0.782659
CD4-positive_alpha-beta-T-cell,0.948140,0.051860
astrocyte,1.000000,0.001000
cardiac-muscle-cell,1.000000,0.001000
endothelial-cell-of-umbilical-vein,0.027803,0.972197
fibroblast-of-dermis,0.999988,0.001000
fibroblast-of-lung,1.000000,0.001000
keratinocyte,0.999997,0.001000
mammary-epithelial-cell,1.000000,0.001000


In [29]:
import matplotlib
qrates = np.array([0.0, 0.9, 0.95, 0.99])
norm = matplotlib.colors.BoundaryNorm(np.array([0.0, 0.91 0.95, 0.99]), 3)
fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: qrates[0::][norm(x)])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3845555027.py, line 3)

In [7]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """
    
    if not ax:
        ax = plt.gca()
        
    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, fraction=0.10, pad=0.04, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)
    
    ax.patch.set(hatch='xx', edgecolor='black')
    

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            if float(text.get_text()) == 0.0:
                text.set_text("")

            texts.append(text)
                
    return texts

In [12]:
colors = matplotlib.colors.LinearSegmentedColormap.from_list('colorblind', sns.color_palette("colorblind", 3), 3)
colors

In [9]:
cad

,cad,CAD
B-cell,0.092117,0.907883
CD14-positive-monocyte,0.217341,0.782659
CD4-positive_alpha-beta-T-cell,0.948140,0.051860
astrocyte,1.000000,0.001000
cardiac-muscle-cell,1.000000,0.001000
endothelial-cell-of-umbilical-vein,0.027803,0.972197
fibroblast-of-dermis,0.999988,0.001000
fibroblast-of-lung,1.000000,0.001000
keratinocyte,0.999997,0.001000
mammary-epithelial-cell,1.000000,0.001000


In [36]:
def plotHeatmaps(dataframe, groupby=None):

    display(dataframe)
    
    colors = matplotlib.colors.LinearSegmentedColormap.from_list('colorblind', sns.color_palette("colorblind", 3), 3)

    labels = groupby

    data     = dataframe[groupby].tolist()
    clusters = list(dataframe[groupby])

    columns = dataframe.index.tolist()
    
    df = pd.DataFrame(columns=[groupby], index=clusters, data=np.array(data).T)

    fig, ax = plt.subplots(1, 1, figsize=(9, 9))

    qrates = np.array([0, 0.9, 0.95, 0.99])
    norm = matplotlib.colors.BoundaryNorm(np.array([0.0, 0.9, 0.95, 0.99]), 3)
    fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: qrates[0::][norm(x)])

    im, _ = heatmap(np.array(data).T.reshape(len(data),1), columns, [groupby], ax=ax, 
                    cmap=colors, norm=norm,
                    cbar_kw=dict(ticks=[0.0, 0.9, 0.95, 0.99], format=fmt),
                    cbarlabel="Confidence interval")

    annotate_heatmap(im, valfmt="{x:.3f}", size=12, textcolors=["white", "white", "white", "white"])

    plt.tight_layout()
    plt.show()
    fig.savefig("figures_paper/CHEERS_enrichment_score.pdf", format='pdf', bbox_inches='tight')

,cad,CAD
B-cell,0.092117,0.907883
CD14-positive-monocyte,0.217341,0.782659
CD4-positive_alpha-beta-T-cell,0.948140,0.051860
astrocyte,1.000000,0.001000
cardiac-muscle-cell,1.000000,0.001000
endothelial-cell-of-umbilical-vein,0.027803,0.972197
fibroblast-of-dermis,0.999988,0.001000
fibroblast-of-lung,1.000000,0.001000
keratinocyte,0.999997,0.001000
mammary-epithelial-cell,1.000000,0.001000


In [37]:
plotHeatmaps(cad, groupby="CAD")